In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, models
from PIL import Image
import os

In [2]:
# Configuración
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "C:/VND_AI/data/no_labels"
batch_size = 32
epochs = 50
learning_rate = 0.001

# Transformaciones de datos (aumentación para SSL)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
# Dataset no etiquetado con dos vistas
class UnlabeledDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith(('jpg', 'png', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image1 = self.transform(image)
            image2 = self.transform(image)
        
        return image1, image2

In [4]:
# Cargar dataset
dataset = UnlabeledDataset(root_dir=data_dir, transform=transform)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
# Modelo SSL con EfficientNet
class SSLModel(nn.Module):
    def __init__(self):
        super(SSLModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.backbone.classifier = nn.Identity()  # Quitar capa final
        self.projector = nn.Sequential(
            nn.Linear(1280, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128)
        )

    def forward(self, x):
        features = self.backbone(x)
        projections = self.projector(features)
        return projections

model = SSLModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

c:\Users\usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Descongelar la red para fine-tuning
for param in model.backbone.parameters():
    param.requires_grad = True

In [7]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images1, images2 in train_loader:
        images1, images2 = images1.to(device), images2.to(device)
        
        optimizer.zero_grad()
        projections1 = model(images1)
        projections2 = model(images2)
        
        # Pérdida de contraste usando similitud de coseno
        loss = -torch.nn.functional.cosine_similarity(projections1, projections2, dim=1).mean()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Imprimir métricas
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/50, Loss: -0.2387
Epoch 2/50, Loss: -0.5966
Epoch 3/50, Loss: -0.8294
Epoch 4/50, Loss: -0.9196
Epoch 5/50, Loss: -0.9481
Epoch 6/50, Loss: -0.9698
Epoch 7/50, Loss: -0.9772
Epoch 8/50, Loss: -0.9824
Epoch 9/50, Loss: -0.9875
Epoch 10/50, Loss: -0.9870
Epoch 11/50, Loss: -0.9799
Epoch 12/50, Loss: -0.9905
Epoch 13/50, Loss: -0.9938
Epoch 14/50, Loss: -0.9911
Epoch 15/50, Loss: -0.9943
Epoch 16/50, Loss: -0.9848
Epoch 17/50, Loss: -0.9959
Epoch 18/50, Loss: -0.9968
Epoch 19/50, Loss: -0.9966
Epoch 20/50, Loss: -0.9976
Epoch 21/50, Loss: -0.9966
Epoch 22/50, Loss: -0.9964


KeyboardInterrupt: 

In [ ]:

# Guardar el modelo SSL
torch.save(model.state_dict(), "models/ssl_efficientnet.pth")